### Unsloth

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 3072 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    # Can select any from the below:
    # "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"
    # "unsloth/Qwen2.5-14B",  "unsloth/Qwen2.5-32B",  "unsloth/Qwen2.5-72B",
    # And also all Instruct versions and Math. Coding verisons!
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


C:\Users\19389\.conda\envs\finetune\lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.3.14: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    NVIDIA GeForce RTX 4060 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.14 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [3]:
from datasets import load_dataset

# 加载单个文件
# dataset = load_dataset("json", data_files=data_path+"output.jsonl")
dataset = load_dataset("json", data_files="outputs/output.jsonl")



In [4]:
from datasets import DatasetDict

# 假设已加载原始数据集 dataset（类型为 DatasetDict）
full_dataset = dataset["train"]  # 提取实际数据部分

# 直接划分：90% 训练，10% 测试
train_test = full_dataset.train_test_split(test_size=0.1, seed=42)

# 组合为 DatasetDict
final_dataset = DatasetDict({
    "train": train_test["train"],
    "test": train_test["test"],
})

# 查看划分结果
print(final_dataset)

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 72
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 8
    })
})


In [5]:
EOS_TOKEN = tokenizer.eos_token  # 必须添加终止符

def formatting_prompts_func(examples):
    texts = []
    # 提取对话内容（假设每条消息按顺序为 system -> user -> assistant）
    try:
        messages = examples['messages']
        system_msg = messages[0]["content"]
        user_msg   = messages[1]["content"]
        assistant_msg = messages[2]["content"]
    # 构建对话模板（根据你的模型需求自定义）
        text = f"[SYSTEM] {system_msg}\n[USER] {user_msg}\n[ASSISTANT] {assistant_msg}{EOS_TOKEN}"
        texts.append(text)
        return texts
    except:
        texts = []
        # print(len(examples['messages']))
        for messages in examples["messages"]:
            # 提取对话内容（假设每条消息按顺序为 system -> user -> assistant）
            system_msg = messages[0]["content"]
            user_msg   = messages[1]["content"]
            assistant_msg = messages[2]["content"]

            # 构建对话模板（根据你的模型需求自定义）
            text = f"[SYSTEM] {system_msg}\n[USER] {user_msg}\n[ASSISTANT] {assistant_msg}{EOS_TOKEN}"
            texts.append(text)
        return texts

message = final_dataset['train'][0]
# system_msg = message["messages"][0]["content"]
# system_msg
print(formatting_prompts_func(message))

['[SYSTEM] ta的五大性格为？\n[USER] {\'emotion\': \'The most possible emotion is 中立/neutral with score 0.48668166995048523\', \'text\': "🎼That I still am with this schedule I have right now, we\'re going to give this a shot to see before we cut down if I cut down videos it\'s going to be.", \'basic_info\': {\'00:00\': {\'Pitch\': \'unknown\', \'Loudness\': \'-9.29\'}, \'00:00.046\': {\'Pitch\': \'C#4\', \'Loudness\': \'-3.45\'}, \'00:00.116\': {\'Pitch\': \'C4\', \'Loudness\': \'-4.46\'}, \'00:00.162\': {\'Pitch\': \'unknown\', \'Loudness\': \'-12.73\'}, \'00:00.232\': {\'Pitch\': \'C#4\', \'Loudness\': \'-4.16\'}, \'00:00.256\': {\'Pitch\': \'A3\', \'Loudness\': \'-3.69\'}, \'00:00.325\': {\'Pitch\': \'G#3\', \'Loudness\': \'-4.63\'}, \'00:00.442\': {\'Pitch\': \'G3\', \'Loudness\': \'-3.79\'}, \'00:00.465\': {\'Pitch\': \'G#3\', \'Loudness\': \'-4.25\'}, \'00:00.488\': {\'Pitch\': \'unknown\', \'Loudness\': \'-4.52\'}, \'00:00.674\': {\'Pitch\': \'G3\', \'Loudness\': \'-2.58\'}, \'00:00.698

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = final_dataset['train'],
    dataset_text_field = "text",
    formatting_func=formatting_prompts_func,
    max_seq_length = max_seq_length,
    dataset_num_proc = 1,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 20,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [7]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4060 Laptop GPU. Max memory = 7.996 GB.
7.309 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 72 | Num Epochs = 3 | Total steps = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176/7,000,000,000 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!